In [16]:
from selenium import webdriver
from datetime import datetime
from time import sleep
import re
import pandas as pd
import locale
#from locale import atof, setlocale, LC_NUMERIC
driver_path_chrome = r'C:\Users\jonat\chromedriver.exe'
driver_path_edge = r'C:\Users\jonat\msedgedriver.exe'
#setlocale(LC_NUMERIC, 'pt_BR.UTF-8')
#locale.setlocale(locale.LC_ALL, "Portuguese_Brazil.1252")
locale.setlocale(locale.LC_ALL, "ko_KR.UTF-8")

'ko_KR.UTF-8'

In [17]:
#driver = webdriver.Edge(driver_path_edge)

In [18]:
def get_price():
    xpath_price = "/html/body/div[2]/section/div[1]/div/div[3]/div[5]/div[1]/div/div[2]/span[1]/strong"
    price_content = driver.find_element_by_xpath(xpath_price).text
    try:
        price_content = price_content.replace('원', '')
    except:
        price_content = price_content
    price = locale.atof(price_content)
    return price

In [19]:
url_list = [
    r'https://www.coupang.com/vp/products/6366671555?isAddedCart=',
    r'https://www.coupang.com/vp/products/5056493492?isAddedCart=',
    r'https://www.coupang.com/vp/products/4978269600?isAddedCart='
]
price_list = []
for url in url_list:
    driver = webdriver.Edge(driver_path_edge)
    driver.get(url)
    act_dict = {
        #'title' : get_title(),
        'price' : get_price(),
        #'price_span0' : get_price_span0(),
        #'price_span2' : get_price_span2(),
        #'price_box' : get_price_box(),
        #'image_link' : get_image_link(),
        'link' : url
        }
    price_list.append(act_dict)
    driver.quit()
    sleep(2)
price_list

[{'price': 69900.0,
  'link': 'https://www.coupang.com/vp/products/6366671555?isAddedCart='},
 {'price': 296900.0,
  'link': 'https://www.coupang.com/vp/products/5056493492?isAddedCart='},
 {'price': 399000.0,
  'link': 'https://www.coupang.com/vp/products/4978269600?isAddedCart='}]

In [20]:
price_list[0]['price'].text

AttributeError: 'float' object has no attribute 'text'